<a href="https://colab.research.google.com/github/jmsarmiento11/csc612m-rabin-karp-CUDA/blob/main/Rabin_Karp_Algorithm_%5BFinal%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Rabin-Karp C++ Program

In [ ]:
%%writefile c_rabinKarp.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <stdbool.h>
#include <limits.h>
#include <time.h>

const int BASE = 256;

// Custom power function to calculate BASE^(m-1) for integers.
unsigned long long int customPow(int base, int exponent) {
    unsigned long long int result = 1;
    while (exponent > 0) {
        if (exponent & 1) {
            result *= base;
        }
        base *= base;
        exponent >>= 1;
    }
    return result;
}

// Function to calculate the hash value for a given substring using prefix sum.
unsigned long long int calculateHash(const char* str, int start, int end) {
    unsigned long long int hashValue = 0;
    for (int i = start; i <= end; ++i) {
        hashValue = (hashValue * BASE + str[i]) % INT_MAX;
    }
    return hashValue;
}

// Function to find the occurrences of a pattern in the text using Rabin-Karp algorithm.
void rabinKarpSearch(const char* text, const char* patterns[], int numPatterns) {
    int n = strlen(text);

    // Start the timer.
    clock_t start = clock();

    // Loop through each pattern.
    for (int k = 0; k < numPatterns; ++k) {
        const char* pattern = patterns[k];
        int m = strlen(pattern);
        unsigned long long int patternHash = calculateHash(pattern, 0, m - 1);

        bool patternFound = false;

        // Loop through the text with a sliding window of length m.
        for (int i = 0; i <= n - m; ++i) {
            unsigned long long int currHash = calculateHash(text, i, i + m - 1);

            // Check if the hash value matches for the current window.
            if (patternHash == currHash) {
                bool found = true;

                // If the hash values match, compare the pattern with the text for an exact match.
                for (int j = 0; j < m; ++j) {
                    if (text[i + j] != pattern[j]) {
                        found = false;
                        break;
                    }
                }

                // If the pattern is found, print its position and set the flag to true.
                if (found) {
                    printf("Pattern \"%s\" found at position: %d\n", pattern, i);
                    patternFound = true;
                }
            }
        }

        // If the pattern is not found in the text, display a message.
        if (!patternFound) {
            printf("Pattern \"%s\" not found in the text.\n", pattern);
        }
    }

    // End the timer and calculate elapsed time.
    clock_t end = clock();
    double elapsed_time = (double)(end - start)*1e6 / CLOCKS_PER_SEC;
    printf("Searching process is done. Elapsed time: %.6f microseconds\n", elapsed_time);
}

int main() {
    FILE* inputFile = fopen("text2.txt", "r");
    if (!inputFile) {
        printf("Error opening the file.\n");
        return 1;
    }

    char* text = NULL;
    size_t len = 0;
    ssize_t read;

    // Read the entire text from the file.
    read = getline(&text, &len, inputFile);
    fclose(inputFile);

    const char* patterns[] = {"flncufrnyd", "yosqvoxrqr", "mldgpkfifa", "wrxxozxwur", "zzxzxrmkrz", "okwtlzpkbz", "bmjnqqnhab", "qigjguyjhq", "aeyvmbfdvd", "soixvqasok" };
    int numPatterns = sizeof(patterns) / sizeof(patterns[0]);

    // Call the Rabin-Karp search function to find all patterns in the text.
    rabinKarpSearch(text, patterns, numPatterns);

    free(text);
    return 0;
}


Overwriting c_rabinKarp.c


In [ ]:
%%shell
g++ c_rabinKarp.c -o c_rabinKarp

In [ ]:
%%shell
./c_rabinKarp

Pattern "flncufrnyd" found at position: 1499
Pattern "yosqvoxrqr" found at position: 3932
Pattern "mldgpkfifa" found at position: 4411
Pattern "wrxxozxwur" found at position: 5338
Pattern "zzxzxrmkrz" found at position: 6985
Pattern "okwtlzpkbz" found at position: 8425
Pattern "bmjnqqnhab" found at position: 8545
Pattern "qigjguyjhq" found at position: 8885
Pattern "aeyvmbfdvd" found at position: 11470
Pattern "soixvqasok" found at position: 12900
Searching process is done. Elapsed time: 10420.000000 microseconds


Rabin-Karp CUDA V1.0

In [ ]:
%%writefile rabinKarp.cu
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

const int BASE = 256;

__device__
unsigned long long int customPow(int base, int exponent) {
    unsigned long long int result = 1;
    while (exponent > 0) {
        if (exponent & 1) {
            result *= base;
        }
        base *= base;
        exponent >>= 1;
    }
    return result;
}

__device__
unsigned long long int calculateHash(const char* str, int start, int end) {
    unsigned long long int hashValue = 0;
    for (int i = start; i <= end; ++i) {
        hashValue = (hashValue * BASE + str[i]) % INT_MAX;
    }
    return hashValue;
}

__global__
void rabinKarpSearch(const char* text, const char* pattern, int patternIndex, int textLength, int patternLength, bool* patternFound) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int numThreads = gridDim.x * blockDim.x;
    int n = textLength;
    int m = patternLength;

    // Calculate the pattern hash value outside the loop since it is constant.
    unsigned long long int patternHash = calculateHash(pattern, 0, m - 1);

    // Loop over the text with grid-stride.
    for (int i = tid; i <= n - m; i += numThreads) {
        unsigned long long int currHash = calculateHash(text, i, i + m - 1);

        if (patternHash == currHash) {
            bool found = true;
            for (int j = 0; j < m; ++j) {
                if (text[i + j] != pattern[j]) {
                    found = false;
                    break;
                }
            }

            if (found) {
                printf("Pattern %d found at position: %d\n", patternIndex, i);
                patternFound[patternIndex] = true;
                return; // Return immediately when pattern is found.
            }
        }
    }
}

int main() {
    FILE* inputFile = fopen("text2.txt", "r");
    if (!inputFile) {
        printf("Error opening the file.\n");
        return 1;
    }

    char* text = NULL;
    size_t len = 0;
    ssize_t read;

    // Read the entire text from the file.
    read = getline(&text, &len, inputFile);
    fclose(inputFile);

    const char* patterns[] = {"flncufrnyd", "yosqvoxrqr", "mldgpkfifa", "wrxxozxwur", "zzxzxrmkrz", "okwtlzpkbz", "bmjnqqnhab", "qigjguyjhq", "aeyvmbfdvd", "soixvqasok" };
    int numPatterns = sizeof(patterns) / sizeof(patterns[0]);

    char* cudaText;
    int textLength = strlen(text);

    // Allocate memory and copy text to the GPU.
    cudaMalloc((void**)&cudaText, textLength);
    cudaMemcpy(cudaText, text, textLength, cudaMemcpyHostToDevice);

    bool patternFound[numPatterns];
    for (int i = 0; i < numPatterns; i++) {
        patternFound[i] = false;
    }

    bool* cudaPatternFound;
    cudaMalloc((void**)&cudaPatternFound, numPatterns * sizeof(bool));
    cudaMemcpy(cudaPatternFound, patternFound, numPatterns * sizeof(bool), cudaMemcpyHostToDevice);

    for (int i = 0; i < numPatterns; i++) {
        int patternLength = strlen(patterns[i]);
        char* cudaPattern;
        cudaMalloc((void**)&cudaPattern, patternLength);
        cudaMemcpy(cudaPattern, patterns[i], patternLength, cudaMemcpyHostToDevice);

        // Launch the kernel with appropriate thread configuration.
        int blockSize = 256;
        int numBlocks = (textLength + blockSize - 1) / blockSize;
        rabinKarpSearch<<<numBlocks, blockSize>>>(cudaText, cudaPattern, i, textLength, patternLength, cudaPatternFound);

        cudaFree(cudaPattern);
    }

    cudaMemcpy(patternFound, cudaPatternFound, numPatterns * sizeof(bool), cudaMemcpyDeviceToHost);
    cudaFree(cudaPatternFound);

    // Print message for patterns that are not found.
    for (int i = 0; i < numPatterns; i++) {
        if (!patternFound[i]) {
            printf("Pattern %d not found.\n", i);
        }
    }

    // Free GPU memory.
    cudaFree(cudaText);

    free(text);

    return 0;
}


Writing rabinKarp.cu


In [ ]:
%%shell
nvcc rabinKarp.cu -o rabinKarp

rabinKarp.cu(70): warning #550-D: variable "read" was set but never used



In [ ]:
%%shell
nvprof ./rabinKarp

==1653== NVPROF is profiling process 1653, command: ./rabinKarp
Pattern 0 found at position: 1499
Pattern 1 found at position: 3932
Pattern 2 found at position: 4411
Pattern 3 found at position: 5338
Pattern 4 found at position: 6985
Pattern 5 found at position: 8425
Pattern 6 found at position: 8545
Pattern 7 found at position: 8885
Pattern 8 found at position: 11470
Pattern 9 found at position: 12900
==1653== Profiling application: ./rabinKarp
==1653== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   97.81%  932.86us        10  93.285us  88.863us  101.95us  rabinKarpSearch(char const *, char const *, int, int, int, bool*)
                    1.98%  18.847us        12  1.5700us  1.3440us  3.5200us  [CUDA memcpy HtoD]
                    0.22%  2.0800us         1  2.0800us  2.0800us  2.0800us  [CUDA memcpy DtoH]
      API calls:   99.42%  513.91ms        12  42.826ms  5.5370us  513.84ms  cudaMalloc
                  

Rabin-Karp CUDA V2.0 (With Thrust Library)

In [ ]:
%%writefile rabinKarp2.cu
#include <stdio.h>
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>

const int BASE = 256;

__device__
unsigned long long int customPow(int base, int exponent) {
    unsigned long long int result = 1;
    while (exponent > 0) {
        if (exponent & 1) {
            result *= base;
        }
        base *= base;
        exponent >>= 1;
    }
    return result;
}

__device__
unsigned long long int calculateHash(const char* str, int start, int end) {
    unsigned long long int hashValue = 0;
    for (int i = start; i <= end; ++i) {
        hashValue = (hashValue * BASE + str[i]) % INT_MAX;
    }
    return hashValue;
}

__global__
void rabinKarpSearch(const char* text, const char* pattern, int patternIndex, int textLength, int patternLength, bool* patternFound) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int numThreads = gridDim.x * blockDim.x;
    int n = textLength;
    int m = patternLength;

    // Calculate the pattern hash value outside the loop since it is constant.
    unsigned long long int patternHash = calculateHash(pattern, 0, m - 1);

    // Loop over the text with grid-stride.
    for (int i = tid; i <= n - m; i += numThreads) {
        unsigned long long int currHash = calculateHash(text, i, i + m - 1);

        if (patternHash == currHash) {
            bool found = true;
            for (int j = 0; j < m; ++j) {
                if (text[i + j] != pattern[j]) {
                    found = false;
                    break;
                }
            }

            if (found) {
                printf("Pattern %d found at position: %d\n", patternIndex, i);
                patternFound[patternIndex] = true;
                return; // Return immediately when pattern is found.
            }
        }
    }
}

int main() {
    FILE* inputFile = fopen("text2.txt", "r");
    if (!inputFile) {
        printf("Error opening the file.\n");
        return 1;
    }

    char* text = NULL;
    size_t len = 0;
    ssize_t read;

    // Read the entire text from the file.
    read = getline(&text, &len, inputFile);
    fclose(inputFile);

    const char* patterns[] = {"flncufrnyd", "yosqvoxrqr", "mldgpkfifa", "wrxxozxwur", "zzxzxrmkrz", "okwtlzpkbz", "bmjnqqnhab", "qigjguyjhq", "aeyvmbfdvd", "soixvqasok" };
    int numPatterns = sizeof(patterns) / sizeof(patterns[0]);

    int textLength = strlen(text);

    // Allocate memory for text on the GPU.
    thrust::device_vector<char> cudaText(text, text + textLength);

    bool patternFound[numPatterns];
    for (int i = 0; i < numPatterns; i++) {
        patternFound[i] = false;
    }

    bool* cudaPatternFound;
    cudaMalloc((void**)&cudaPatternFound, numPatterns * sizeof(bool));
    cudaMemcpy(cudaPatternFound, patternFound, numPatterns * sizeof(bool), cudaMemcpyHostToDevice);

    for (int i = 0; i < numPatterns; i++) {
        int patternLength = strlen(patterns[i]);
        char* cudaPattern;
        cudaMalloc((void**)&cudaPattern, patternLength);
        cudaMemcpy(cudaPattern, patterns[i], patternLength, cudaMemcpyHostToDevice);

        // Launch the kernel with appropriate thread configuration.
        int blockSize = 256;
        int numBlocks = (textLength + blockSize - 1) / blockSize;
        rabinKarpSearch<<<numBlocks, blockSize>>>(thrust::raw_pointer_cast(cudaText.data()), thrust::raw_pointer_cast(cudaPattern), i, textLength, patternLength, cudaPatternFound);

        cudaFree(cudaPattern);
    }

    cudaMemcpy(patternFound, cudaPatternFound, numPatterns * sizeof(bool), cudaMemcpyDeviceToHost);
    cudaFree(cudaPatternFound);

    // Print message for patterns that are not found.
    for (int i = 0; i < numPatterns; i++) {
        if (!patternFound[i]) {
            printf("Pattern %d not found.\n", i);
        }
    }

    // Free GPU memory.
    cudaText.clear();

    free(text);

    return 0;
}


Writing rabinKarp2.cu


In [ ]:
%%shell
nvcc rabinKarp2.cu -o rabinKarp2

rabinKarp2.cu(70): warning #550-D: variable "read" was set but never used



In [ ]:
%%shell
nvprof ./rabinKarp2

==4385== NVPROF is profiling process 4385, command: ./rabinKarp2
Pattern 0 found at position: 1499
Pattern 1 found at position: 3932
Pattern 2 found at position: 4411
Pattern 3 found at position: 5338
Pattern 4 found at position: 6985
Pattern 5 found at position: 8425
Pattern 6 found at position: 8545
Pattern 7 found at position: 8885
Pattern 8 found at position: 11470
Pattern 9 found at position: 12900
==4385== Profiling application: ./rabinKarp2
==4385== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   97.80%  939.61us        10  93.960us  89.215us  102.50us  rabinKarpSearch(char const *, char const *, int, int, int, bool*)
                    1.99%  19.104us        12  1.5920us  1.3440us  4.0320us  [CUDA memcpy HtoD]
                    0.22%  2.0800us         1  2.0800us  2.0800us  2.0800us  [CUDA memcpy DtoH]
      API calls:   99.45%  319.05ms        12  26.587ms  4.7170us  318.98ms  cudaMalloc
                